In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/my-dataset/sample_test.csv
/kaggle/input/my-dataset/final_cleaned_dataset.csv
/kaggle/input/my-dataset/sample_test_out.csv
/kaggle/input/my-dataset/train_embeddings_dict_effnetv2.joblib
/kaggle/input/my-dataset/train_embeddings_efficientnetv2s.npy
/kaggle/input/my-dataset/train.csv
/kaggle/input/my-dataset/test.csv


In [17]:
import pandas as pd
import numpy as np
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

In [44]:
#train file
path = "/kaggle/input/my-dataset/train.csv"
df_train = pd.read_csv(path)
df_train.shape

(75000, 4)

In [45]:
embeddings = np.load('/kaggle/input/effnetb7-train/train_embeddings_efficientnetb7s.npy')

In [46]:
embeddings.shape

(75000, 2560)

In [47]:
# embeddings.shape = (n_rows, embedding_dim)
embeddings_df = pd.DataFrame(
    embeddings,
    columns=[f"dim_{i}" for i in range(embeddings.shape[1])]
)

# Add all columns at once
df_train = pd.concat([df_train.reset_index(drop=True),
                      embeddings_df.reset_index(drop=True)],
                     axis=1)

In [48]:
df_train.shape

(75000, 2564)

In [49]:
df_cleaned = pd.read_csv("/kaggle/input/my-dataset/final_train_dataset.csv")   # after dropping rows & removing outliers

In [39]:
df_cleaned.head()

,sample_id,catalog_content,image_link,price,item_name,brand_name,bullet_points,product_description,value,unit
0,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,"Salerno Cookies, The Original Butter Cookies, ...",Salerno,['original butter cookies classic butter cooki...,NaN,32.00,Ounce
1,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,"Bear Creek Hearty Soup Bowl, Creamy Chicken wi...",Bear,['loaded hearty long grain wild rice vegetable...,NaN,11.40,Ounce
2,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,Judee’s Blue Cheese Powder 11.25 oz - Gluten-F...,Judee’s,['add favorite appetizers dips spreads use sea...,judees powdered blue cheese cheddar cheese pow...,11.25,Ounce
3,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,"kedem Sherry Cooking Wine, 12.7 Ounce - 12 per...",kedem,['kedem sherry cooking wine ounce per case'],NaN,12.00,Count
4,9259,"Item Name: Member's Mark Member's Mark, Basil,...",https://m.media-amazon.com/images/I/81nw0HXpCR...,18.50,"Member's Mark Member's Mark, Basil, 6.25 oz",Member's,['green herb italian staple great mixed oregan...,NaN,6.25,ounce


In [50]:
df_with_embeddings = df_train.merge(
    df_cleaned,
    on="sample_id",     # or filename, or id
    how="inner"
)

In [51]:
df_with_embeddings.shape

(56102, 2573)

In [52]:
df_with_embeddings.head()

,sample_id,catalog_content_x,image_link_x,price_x,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,...,dim_2559,catalog_content_y,image_link_y,price_y,item_name,brand_name,bullet_points,product_description,value,unit
0,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,-0.180298,0.145386,0.073486,-0.096497,-0.217041,-0.069214,...,-0.160645,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,"Salerno Cookies, The Original Butter Cookies, ...",Salerno,['original butter cookies classic butter cooki...,NaN,32.00,Ounce
1,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,-0.150391,0.124817,0.296631,0.250977,-0.130981,0.510254,...,-0.087036,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,"Bear Creek Hearty Soup Bowl, Creamy Chicken wi...",Bear,['loaded hearty long grain wild rice vegetable...,NaN,11.40,Ounce
2,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,0.033569,-0.083130,0.494873,0.045990,-0.038818,0.037628,...,-0.164307,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,Judee’s Blue Cheese Powder 11.25 oz - Gluten-F...,Judee’s,['add favorite appetizers dips spreads use sea...,judees powdered blue cheese cheddar cheese pow...,11.25,Ounce
3,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,0.133423,-0.113464,-0.142334,-0.209351,-0.127808,-0.135376,...,-0.111572,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,"kedem Sherry Cooking Wine, 12.7 Ounce - 12 per...",kedem,['kedem sherry cooking wine ounce per case'],NaN,12.00,Count
4,9259,"Item Name: Member's Mark Member's Mark, Basil,...",https://m.media-amazon.com/images/I/81nw0HXpCR...,18.50,0.051025,0.270752,0.371094,0.007622,-0.085815,0.036774,...,-0.140381,"Item Name: Member's Mark Member's Mark, Basil,...",https://m.media-amazon.com/images/I/81nw0HXpCR...,18.50,"Member's Mark Member's Mark, Basil, 6.25 oz",Member's,['green herb italian staple great mixed oregan...,NaN,6.25,ounce


In [53]:
cols_to_drop = [col for col in df_with_embeddings.columns if col.endswith('_y')]
df_with_embeddings = df_with_embeddings.drop(columns=cols_to_drop)

# Optionally, rename *_x back to original names
df_with_embeddings.columns = [
    col.replace('_x', '') for col in df_with_embeddings.columns
]

In [54]:
df_with_embeddings.shape

(56102, 2570)

In [55]:
df_with_embeddings.head()

,sample_id,catalog_content,image_link,price,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,...,dim_2556,dim_2557,dim_2558,dim_2559,item_name,brand_name,bullet_points,product_description,value,unit
0,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,-0.180298,0.145386,0.073486,-0.096497,-0.217041,-0.069214,...,-0.012589,0.092224,-0.121948,-0.160645,"Salerno Cookies, The Original Butter Cookies, ...",Salerno,['original butter cookies classic butter cooki...,NaN,32.00,Ounce
1,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,-0.150391,0.124817,0.296631,0.250977,-0.130981,0.510254,...,-0.137573,0.005196,-0.067078,-0.087036,"Bear Creek Hearty Soup Bowl, Creamy Chicken wi...",Bear,['loaded hearty long grain wild rice vegetable...,NaN,11.40,Ounce
2,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,0.033569,-0.083130,0.494873,0.045990,-0.038818,0.037628,...,0.014023,0.005238,-0.140747,-0.164307,Judee’s Blue Cheese Powder 11.25 oz - Gluten-F...,Judee’s,['add favorite appetizers dips spreads use sea...,judees powdered blue cheese cheddar cheese pow...,11.25,Ounce
3,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,0.133423,-0.113464,-0.142334,-0.209351,-0.127808,-0.135376,...,0.050873,-0.165405,0.071655,-0.111572,"kedem Sherry Cooking Wine, 12.7 Ounce - 12 per...",kedem,['kedem sherry cooking wine ounce per case'],NaN,12.00,Count
4,9259,"Item Name: Member's Mark Member's Mark, Basil,...",https://m.media-amazon.com/images/I/81nw0HXpCR...,18.50,0.051025,0.270752,0.371094,0.007622,-0.085815,0.036774,...,-0.027893,0.325684,-0.196411,-0.140381,"Member's Mark Member's Mark, Basil, 6.25 oz",Member's,['green herb italian staple great mixed oregan...,NaN,6.25,ounce


In [56]:
df_with_embeddings.to_csv('/kaggle/working/final_train_plus_effnetb7.csv')